In [1]:
import os
import pandas as pd
import numpy as np
import regex as re
from datetime import datetime

pd.set_option('display.max_columns', 100)

In [2]:
current_dir = os.getcwd()
NOAA_dir = current_dir + r'\NOAA Weather Data'
EPA_dir = current_dir + r'\EPA Ozone Data'

In [3]:
zip_files = os.listdir(EPA_dir + r'\Raw EPA Data')
zip_files

['hourly_44201_1989.zip',
 'hourly_44201_1990.zip',
 'hourly_44201_1991.zip',
 'hourly_44201_1992.zip',
 'hourly_44201_1993.zip',
 'hourly_44201_1994.zip',
 'hourly_44201_1995.zip',
 'hourly_44201_1996.zip',
 'hourly_44201_1997.zip',
 'hourly_44201_1998.zip',
 'hourly_44201_1999.zip',
 'hourly_44201_2000.zip',
 'hourly_44201_2001.zip',
 'hourly_44201_2002.zip',
 'hourly_44201_2003.zip']

In [4]:
def create_ozone_id(statelist, countylist, sitelist):
    """
    Helper function for creating ozoneID's based on an EPA dataset
    
    returns: 3 lists appended together into tuples to be added into a column
    """
    return [(a,b,c) for a, b, c in zip(statelist, countylist, sitelist)]

def append_ozone_id(ozone_df):
    """
    Creates a copy of the dataframe and adds a new column that concatenates the state code, county code, and site number
    into a tuple to make an individual identifier for each ozone reporting location in that year. 
    
    returns: dataframe with ozoneID
    """
    temp = ozone_df.copy() 
    temp["ozoneID"] = create_ozone_id(temp["State Code"], temp["County Code"], temp["Site Num"])
    return temp

In [5]:
time_period = ['09:00', '10:00', '11:00', '12:00', '13:00', '14:00', 
                      '15:00', '16:00', '17:00', '18:00', '19:00', '20:00']

def monitor_day_filter(grouped_dataframe):
    """
    Used in split-apply-combine after grouping by monitor-day in order to filter out days that have 
    less than 9 observations in the hours from 9am to 9pm
    """
    times = grouped_dataframe["Time Local"]
    indicator = [time in time_period for time in times]
    return sum(indicator) >= 9

## Construct 2 measures of ozone concentrations at the monitor-day level

- Daily Maximum: Groupby date, then return the maximum of that day

- Daily 8-hour Maximum: Groupby date, then average hours 0-8, 8-4, 4-12, return the maximum of that 

- Disqualify all monitor-days for which observations are not recorded for at least 9 hours between 9AM and 9PM, disqualify all monitors that have less than 75% of the days recorded from June1 to August 31,

- Disqualify monitors that are in counties close to other counties that have more stringent regulation (?????) 

Do this in one group by, write an apply function to return both of these as a new data frame per date. 

In [6]:
def classify_time(time_str):
    time_int = int(time_str[0:2])
    if time_int < 8:
        return 1
    elif 8 <= time_int < 16:
        return 2
    elif  16 <= time_int:
        return 3
    else:
        raise ValueError('Time Local date was invalid?') 

def calculate_max_8hrmax(grouped_dataframe):
    """
    Used in split-apply-combine after grouping by monitor-day - calculate the maximum of the day as well as the 
    8 hour maximum - 8 hour maximum 
    """
       
    grouped_dataframe["Time Chunk"] = grouped_dataframe["Time Local"].apply(classify_time)
    
    samples_all = grouped_dataframe["Sample Measurement"]
    
    mean_1 = np.mean(grouped_dataframe.loc[grouped_dataframe["Time Chunk"] == 1, "Sample Measurement"])
    mean_2 = np.mean(grouped_dataframe.loc[grouped_dataframe["Time Chunk"] == 2, "Sample Measurement"])
    mean_3 = np.mean(grouped_dataframe.loc[grouped_dataframe["Time Chunk"] == 3, "Sample Measurement"])
    
    daily_mean = np.mean(samples_all)
    eight_hour_mean = max(mean_1, mean_2, mean_3)
    
    output = grouped_dataframe.iloc[[0],:]
    
    output["Daily Mean Ozone"] = daily_mean
    output["Daily 8hr Mean Ozone"] = eight_hour_mean
    return output
                        

In [7]:
@np.vectorize
def is_summer(date_str):
    """
    Checks if a dat str (in the format of the tablse) is in the summer 
    """
    month = int(re.search(r'\d{4}-(\d{2})-\d{2}', date_str).group(1))
    return 6 <= month <= 8


def monitor_year_filter(grouped_ozone_id):
    """ 
    Used in split-apply-combine after grouping by monitor in order to filter out entire monitors that do not have 
    observations in 25% or more during the summer ozone months Do this after calculating the maximum and 8hr maximums 
    
    DO THIS BEFORE APPLYING MAX DATE BECAUSE MAX DATE SLOW AS HELL 
    """
    # 92 days between June 1 and August 31, need 75% observations or more == only accept if greater than or equal to 69
    dates = grouped_ozone_id["Date Local"].unique()
    return sum(is_summer(dates)) >= 69
   
    

## For Loop Instructions 
- Read the csv, then append ozone_id to get the ozone_Ids
- First group by date and ozone_id and filter out monitor-dates using monitor-day-filter 
    - This will net us with a dataframe that only has monitor dates for enough observations in the 9 hours  
- Apply the calculate max-8hrmax function to grouped monitor-days 
    - This will result with each day ending up justb being a single observation with 2 new columns. 
- Now group by monitor and look at whether or not this year had enough values, return final data frame after filtering

In [8]:
zip_files = os.listdir(EPA_dir + r'\Raw EPA Data')
zip_files

for filename in zip_files: 
    
    year = filename[13:17]
    
    if f"filtered_{year}_EPA.csv" in os.listdir(EPA_dir + r'\Filtered EPA Data'):
        print(f"Skipping {year}. already done")
        continue
    else: 
        print(f"working on {year}")
        filepath = EPA_dir + r"\\Raw EPA Data\\" + filename 
        temp = pd.read_csv(filepath)
        print("read csv done!")
        
        temp = append_ozone_id(temp)
        print(f"finished appending for {filename}")
        
        temp = temp.groupby(["ozoneID", "Date Local"]).filter(monitor_day_filter)
        print(f"finished dayfilter for {filename}")
        
        temp = temp.groupby("ozoneID").filter(monitor_year_filter)
        print(f"finished yearfilter for {filename}")
        
        temp = temp.groupby(["ozoneID", "Date Local"], group_keys=False).apply(calculate_max_8hrmax)
        print(f"finished {filename}, exporting to csv!")

        path = EPA_dir + r'\Filtered EPA Data\filtered_' + year + "_EPA.csv"
        temp.to_csv(path)

Skipping 1989. already done
Skipping 1990. already done
Skipping 1991. already done
Skipping 1992. already done
Skipping 1993. already done
Skipping 1994. already done
Skipping 1995. already done
Skipping 1996. already done
Skipping 1997. already done
Skipping 1998. already done
Skipping 1999. already done
Skipping 2000. already done
Skipping 2001. already done
Skipping 2002. already done
Skipping 2003. already done


Note: (13, 85, 1) has 48 counts for some reason - it's just duplicated values can ignore because not gonna affect mean and 8hr mean

## Filtering Counties and Getting Only Summer Months to Make Table


### Creating the lists of state_code county_code pairs that need to be deleted  

In [9]:
county_list = pd.read_stata(current_dir + "\Author Data\AER20090377_CountyList.dta")
county_list["fips"] = county_list["fips"].astype(int)
county_list = county_list.rename(columns = {"state_code": "State Code", "county_code":"County Code"})
county_list = county_list.drop(columns = "county_desc")
county_list.head()

,fips,State Code,County Code,population,min_latitude,max_latitude,min_longitude,max_longitude
0,1001,1,1,43671,32.307499,32.706669,-86.919441,-86.411110
1,1003,1,3,140415,30.221111,31.317778,-88.028610,-87.371391
2,1005,1,5,29038,31.617500,32.148056,-85.747780,-85.050278
3,1007,1,7,20826,32.830002,33.245834,-87.421387,-86.875832
4,1009,1,9,51024,33.764999,34.259998,-86.963608,-86.302780


In [10]:
neighbor_list = pd.read_stata(current_dir + "\Author Data\AER20090377_NeighborData.dta")
neighbor_list.head()

,fips,treated_neighbor
0,1003,1.0
1,1033,0.0
2,1055,0.0
3,1061,0.0
4,1069,0.0


In [11]:
# If deleting too many values, then make this so that it only counts the neighbors that are == 1
neighbor_fips = neighbor_list["fips"].values

In [12]:
def get_urban_df(year):
    """
    Returns the dataframe of the urban designations as extracted from "Getting Urban Designation.ipynb"|
    """
    assert 1989 <= year <= 2003, "Bad year input"
    df = pd.read_csv(current_dir + r"\Author Data\county_urban_designation\county_urban_designation_" + str(year) + ".csv")
    df["urban"] = [int(text[1]) for text in df["urban"]]
    return df
    

### Filtering out the above counties for each file, selecting only the summer months, and then also storing necessary data to create the summary table, also adding rural/urban/suburban

- First add all of county_list data to the normal data
- Filter based on that 
- Apply county filter from above
- Store the number of remaining ozone_locations
- Store the number of monitor-days left
- Count how many are rural/urban 

In [13]:
statistics = {}

In [14]:
filtered_dir_list = os.listdir(EPA_dir + r'\Filtered EPA Data')
filtered_files = [file for file in filtered_dir_list if file.endswith(".csv")]
filtered_files

['filtered_1989_EPA.csv',
 'filtered_1990_EPA.csv',
 'filtered_1991_EPA.csv',
 'filtered_1992_EPA.csv',
 'filtered_1993_EPA.csv',
 'filtered_1994_EPA.csv',
 'filtered_1995_EPA.csv',
 'filtered_1996_EPA.csv',
 'filtered_1997_EPA.csv',
 'filtered_1998_EPA.csv',
 'filtered_1999_EPA.csv',
 'filtered_2000_EPA.csv',
 'filtered_2001_EPA.csv',
 'filtered_2002_EPA.csv',
 'filtered_2003_EPA.csv']

In [24]:
filtered_files = ['filtered_1989_EPA.csv',
 'filtered_1990_EPA.csv',
 'filtered_1991_EPA.csv',
 'filtered_1992_EPA.csv',
 'filtered_1993_EPA.csv',
 'filtered_1994_EPA.csv',
 'filtered_1995_EPA.csv',
 'filtered_1996_EPA.csv',
 'filtered_1997_EPA.csv',
 'filtered_1998_EPA.csv',
 'filtered_1999_EPA.csv',
 'filtered_2000_EPA.csv',
 'filtered_2001_EPA.csv',
 'filtered_2002_EPA.csv',
 'filtered_2003_EPA.csv']

In [25]:
summer_output_folder_path = EPA_dir + r'\Filtered EPA Data\onlysummer\\' 
second_level_cleaning_folder_path = EPA_dir + r'\Filtered EPA Data\second_cleaning\\' 

In [26]:
county_list.iloc[:,0:3]

,fips,State Code,County Code
0,1001,1,1
1,1003,1,3
2,1005,1,5
3,1007,1,7
4,1009,1,9
...,...,...,...
3270,80028,80,28
3271,80029,80,29
3272,80030,80,30
3273,80031,80,31


In [27]:
for df_name in filtered_files:
    
    year = int(df_name[9:13])
    
    if False:
#     if r"summer_only_EPA_" + str(year) + ".csv" in os.listdir(summer_output_folder_path):
        print(r"summer_only_EPA_" + str(year) + ".csv already exists, skipping")
        continue
    else:
        
        year = int(df_name[9:13])
        
        print("working on " + str(year))
        temp = pd.read_csv(EPA_dir + r'\Filtered EPA Data\\' + df_name, low_memory = False)

        
        # IF THERE IS CANADA, DELETE THOSE ROWS LMAO 
        
        if "CC" in temp["State Code"].unique(): 
            temp = temp[temp["State Code"] != "CC"]
            temp["State Code"] = temp["State Code"].astype(int)
            
        
        # add information from county_list file -- add fips 
        temp = temp.merge(county_list.iloc[:,0:3], on = ["State Code", "County Code"]) # now we have a bunch of extra info but that's okay 

        # remove counties that are in the neighbors list 
        neighbor_indicator = [county_fip not in neighbor_fips for county_fip in temp["fips"]]
        temp = temp[neighbor_indicator] 


        # add urban designation - for missing observations with NaN, fill with zero.
        # 3 = suburban, 2 = rural, 1 = urban
        temp = temp.merge(get_urban_df(year), on = "ozoneID", how = "left").fillna({"urban":0})     
        
        temp = temp.drop(columns = ["Parameter Code", "POC", "Datum", "Parameter Name", "Time Local", "Date GMT", "Time GMT", "Units of Measure", "Uncertainty", "Qualifier", "Method Type", "Method Code", "Method Name", "MDL", "Date of Last Change", "Time Chunk"])

        temp.to_csv(second_level_cleaning_folder_path + r"condensed_EPA_" + str(year) + ".csv")

#         # select only observations that are in the months of June, July, August 
#         summer_indicator = [int(date[5:7]) in [6, 7, 8] for date in temp["Date Local"]]
#         summer_only = temp[summer_indicator]

#         summer_only.to_csv(summer_output_folder_path + r"summer_only_EPA_" + str(year) + ".csv")

        #get statistics

#         num_obs = summer_only.shape[0]
#         num_counties = len(summer_only["fips"].unique())
#         num_monitors = len(summer_only["ozoneID"].unique())
#         urban_0_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 0)
#         urban_1_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 1)
#         urban_2_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 2)
#         urban_3_count = sum(summer_only.groupby("ozoneID")["urban"].unique() == 3)

#         statistics[year] = {"Observations":num_obs,
#                             "Counties":num_counties,
#                             "Total Monitors":num_monitors,
#                             "Urban_1":urban_1_count,
#                             "Urban_2":urban_2_count,
#                             "Urban_3":urban_3_count,
#                             "Urban_0":urban_0_count,
#                            }
            
    

working on 1989


<ipython-input-27-84b04f642fc5>:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "CC" in temp["State Code"].unique():


working on 1990
working on 1991
working on 1992
working on 1993
working on 1994
working on 1995
working on 1996
working on 1997
working on 1998
working on 1999
working on 2000
working on 2001
working on 2002
working on 2003


In [27]:
# summary_table = pd.DataFrame(statistics).T
# summary_table.to_csv("summary_table2.csv")
# summary_table

,Observations,Counties,Total Monitors,Urban_1,Urban_2,Urban_3,Urban_0
1989,63838,424,728,153,328,245,2
1990,69599,467,790,157,331,269,33
1991,72745,483,822,151,340,298,33
1992,73370,482,828,155,338,301,34
1993,76311,501,857,167,352,302,36
1994,78827,511,884,163,361,318,42
1996,80301,504,897,165,364,331,37
1997,82747,515,923,166,375,337,45
1998,84762,524,945,165,385,345,50
1999,85814,523,956,168,393,345,50


In [28]:
pd.read_csv("summary_table2.csv", index_col = 0)

,Observations,Counties,Total Monitors,Urban_1,Urban_2,Urban_3,Urban_0
1989,63838,424,728,153,328,245,2
1990,69599,467,790,157,331,269,33
1991,72745,483,822,151,340,298,33
1992,73370,482,828,155,338,301,34
1993,76311,501,857,167,352,302,36
1994,78827,511,884,163,361,318,42
1996,80301,504,897,165,364,331,37
1997,82747,515,923,166,375,337,45
1998,84762,524,945,165,385,345,50
1999,85814,523,956,168,393,345,50
